# Function Calling -- Format JSON

Function calling is my favourite, open secret about modern LLMs: They can be used to communicate with other systems. 

How do we do that? By parsing your information into a JSON, which can then be used to interact with your APIs. There are more straight forward use cases for JSON output as well e.g. parsing a text into structured data.

The remainder of this notebook is directly from the [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb)


This notebook covers how to use the Chat Completions API in combination with external functions to extend the capabilities of GPT models.

`tools` is an optional parameter in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to the provided specifications. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

Within the `tools` parameter, if the `functions` parameter is provided then by default the model will decide when it is appropriate to use one of the functions. The API can be forced to use a specific function by setting the `tool_choice` parameter to `{"name": "<insert-function-name>"}`. The API can also be forced to not use any function by setting the `tool_choice` parameter to `"none"`. If a function is used, the output will contain `"finish_reason": "function_call"` in the response, as well as a `tool_choice` object that has the name of the function and the generated function arguments.

### Overview

This notebook contains the following 2 sections:

- **How to generate function arguments:** Specify a set of functions and use the API to generate function arguments.
- **How to call functions with model generated arguments:** Close the loop by actually executing functions with model generated arguments.

## How to generate function arguments

In [2]:
import json

from dotenv import load_dotenv
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_random_exponential
from termcolor import colored

load_dotenv()
GPT_MODEL = "gpt-3.5-turbo-0613"
client = OpenAI()

### Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [3]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [4]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


### Basic concepts

Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [6]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content='Sure, could you please provide me with the location?', role='assistant', function_call=None, tool_calls=None)

Once we provide the missing information, it will generate the appropriate function arguments for us.

In [7]:
messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_oNegav2r7A6xPALkwkolEMLE', function=Function(arguments='{\n  "location": "Glasgow, Scotland",\n  "format": "celsius"\n}', name='get_current_weather'), type='function')])

By prompting it differently, we can get it to target the other function we've told it about.

In [8]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content='Sure, I can help you with that. Please provide the number of days you want to forecast.', role='assistant', function_call=None, tool_calls=None)

Once again, the model is asking us for clarification because it doesn't have enough information yet. In this case it already knows the location for the forecast, but it needs to know how many days are required in the forecast.

In [9]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0]


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_OV9VdLnHnM8PEkivGHtQVmrT', function=Function(arguments='{\n  "location": "Glasgow, Scotland",\n  "format": "celsius",\n  "num_days": 5\n}', name='get_n_day_weather_forecast'), type='function')]))

#### Forcing the use of specific functions or no function

We can force the model to use a specific function, for example get_n_day_weather_forecast by using the function_call argument. By doing so, we force the model to make assumptions about how to use it.

In [10]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
chat_response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_WaBjmbCPPcR9Ni8y1ZNyGhUR', function=Function(arguments='{\n  "location": "Toronto, Canada",\n  "format": "celsius",\n  "num_days": 1\n}', name='get_n_day_weather_forecast'), type='function')])

In [11]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_sfyfwJERT0XkfZ8GGAKpF18v', function=Function(arguments='{\n  "location": "Toronto, Canada",\n  "format": "celsius"\n}', name='get_current_weather'), type='function')])

We can also force the model to not use a function at all. By doing so we prevent it from producing a proper function call.

In [12]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice="none"
)
chat_response.choices[0].message


ChatCompletionMessage(content='{ "location": "Toronto, Canada", "format": "celsius" }', role='assistant', function_call=None, tool_calls=None)

### Parallel Function Calling

Newer models like gpt-4-1106-preview or gpt-3.5-turbo-1106 can call multiple functions in one turn.

In [13]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model='gpt-3.5-turbo-1106'
)

assistant_message = chat_response.choices[0].message.tool_calls
assistant_message

[ChatCompletionMessageToolCall(id='call_5aJObKpRPbGWotcboJbuayjS', function=Function(arguments='{"location": "San Francisco", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function'),
 ChatCompletionMessageToolCall(id='call_Sp92VXTDYrkpau7p5UwCbiRc', function=Function(arguments='{"location": "Glasgow", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function')]

## Next Steps

Checkout other [notebook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_for_knowledge_retrieval.ipynb) that demonstrates how to use the Chat Completions API and functions for knowledge retrieval to interact conversationally with a knowledge base.

# Using Function Calling with Pydantic

Pydantic is a data validation and settings management using Python type annotations. It is a library that allows you to validate data and settings using Python type annotations. It is a great library to use with FastAPI, but it can be used with any Python project. We use instructor (https://jxnl.github.io/instructor) to demonstrate how to use Pydantic with Function Calling.

As an illustration, we will use a complex task: Graph Construction -- but a simple example, so that you can self verify the errors with the model outputs. 

## Graph Construction

In this guide, we demonstrate how to extract and resolve entities from a sample legal contract. Then, we visualize these entities and their dependencies as an entity graph

In [1]:
from pydantic import BaseModel, Field
from typing import List


class Property(BaseModel):
    key: str
    value: str
    resolved_absolute_value: str


class Entity(BaseModel):
    id: int = Field(
        ...,
        description="Unique identifier for the entity, used for deduplication, design a scheme allows multiple entities",
    )
    subquote_string: List[str] = Field(
        ...,
        description="Correctly resolved value of the entity, if the entity is a reference to another entity, this should be the id of the referenced entity, include a few more words before and after the value to allow for some context to be used in the resolution",
    )
    entity_title: str
    properties: List[Property] = Field(
        ..., description="List of properties of the entity"
    )
    dependencies: List[int] = Field(
        ...,
        description="List of entity ids that this entity depends  or relies on to resolve it",
    )

class DocumentExtraction(BaseModel):
    entities: List[Entity] = Field(
        ...,
        description="Body of the answer, each fact should be a separate object with a body and a list of sources",
    )

In [3]:
import instructor
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
# Apply the patch to the OpenAI client
# enables response_model keyword
client = instructor.patch(OpenAI())


def ask_ai(content) -> DocumentExtraction:
    return client.chat.completions.create(
        model="gpt-4",
        response_model=DocumentExtraction,
        messages=[
            {
                "role": "system",
                "content": "Extract and resolve a list of entities from the following document:",
            },
            {
                "role": "user",
                "content": content,
            },
        ],
    )  # type: ignore

In [6]:
from graphviz import Digraph


def generate_html_label(entity: Entity) -> str:
    rows = [
        f"<tr><td>{prop.key}</td><td>{prop.resolved_absolute_value}</td></tr>"
        for prop in entity.properties
    ]
    table_rows = "".join(rows)
    return f"<<table border='0' cellborder='1' cellspacing='0'><tr><td colspan='2'><b>{entity.entity_title}</b></td></tr>{table_rows}</table>>"


def generate_graph(data: DocumentExtraction):
    dot = Digraph(comment="Entity Graph", node_attr={"shape": "plaintext"})

    for entity in data.entities:
        label = generate_html_label(entity)
        dot.node(str(entity.id), label)

    for entity in data.entities:
        for dep_id in entity.dependencies:
            dot.edge(str(entity.id), str(dep_id))

    dot.render("entity.gv", view=True)

In [8]:
content = """
Sample Legal Contract
Agreement Contract

This Agreement is made and entered into on 2020-01-01 by and between Company A ("the Client") and Company B ("the Service Provider").

Article 1: Scope of Work

The Service Provider will deliver the software product to the Client 30 days after the agreement date.

Article 2: Payment Terms

The total payment for the service is $50,000.
An initial payment of $10,000 will be made within 7 days of the the signed date.
The final payment will be due 45 days after [SignDate].

Article 3: Confidentiality

The parties agree not to disclose any confidential information received from the other party for 3 months after the final payment date.

Article 4: Termination

The contract can be terminated with a 30-day notice, unless there are outstanding obligations that must be fulfilled after the [DeliveryDate].
"""  # Your legal contract here
model = ask_ai(content)

In [11]:
generate_graph(model)

In [12]:
example_content = """
# Technical Document: User Profile Service with Horizontal Scaling

## Overview

The User Profile Service is a microservice designed to manage user profiles in a large-scale web application. It provides functionalities such as creating, updating, retrieving, and deleting user profile information. Given the dynamic nature of user interactions and the potential for high traffic, this service is designed with horizontal scaling capabilities to ensure high availability, resilience, and consistent performance under varying loads.

## Service Description

### Functional Requirements

1. **Create Profile**: Allows creating a new user profile with details like name, email, and preferences.
2. **Update Profile**: Permits modifications to existing profiles.
3. **Retrieve Profile**: Enables fetching profile details for a given user ID.
4. **Delete Profile**: Supports removal of a user profile from the system.

### Non-Functional Requirements

1. **Scalability**: The service must scale horizontally to handle spikes in traffic.
2. **Performance**: Response times should remain consistent under load.
3. **Availability**: Designed for 99.99% uptime.
4. **Security**: Ensure data protection and secure access.

## System Architecture

### Components

1. **API Gateway**: Serves as the entry point for all client requests, routing them to the appropriate service instance.
2. **Service Instances**: Multiple instances of the User Profile Service, each capable of handling requests independently.
3. **Load Balancer**: Distributes incoming requests evenly across service instances to ensure load distribution and high availability.
4. **Database Cluster**: A horizontally scalable database that stores user profile information. It ensures data consistency and high availability through replication and sharding techniques.
5. **Cache Layer**: An in-memory cache (e.g., Redis) to reduce database load and improve response times for frequently accessed data.

### Data Flow

1. A client request hits the API Gateway.
2. The request is forwarded to the Load Balancer.
3. The Load Balancer selects a Service Instance based on load distribution algorithms.
4. The Service Instance processes the request. It may interact with the Cache Layer or the Database Cluster as needed.
5. The response is sent back to the client following the reverse path.

## Horizontal Scaling Strategy

### Service Instances

- **Auto-Scaling**: Service instances are scaled horizontally based on CPU and memory usage metrics. When thresholds are exceeded, new instances are automatically spawned across multiple servers or cloud infrastructure.

### Database Cluster

- **Sharding**: The database is partitioned into shards, each holding a subset of the data. This distributes the load and enables the database to grow horizontally.
- **Replication**: Each shard is replicated across multiple nodes to ensure data availability and fault tolerance.

### Cache Layer

- The cache layer is designed to scale out by adding more nodes, which allows for distributing cache data and handling more concurrent connections.

## Technologies Used

- **API Gateway**: NGINX, Amazon API Gateway
- **Service Implementation**: Node.js, Express.js
- **Load Balancer**: HAProxy, AWS Elastic Load Balancing
- **Database**: MongoDB with sharding, Cassandra
- **Cache**: Redis Cluster
- **Containerization**: Docker
- **Orchestration**: Kubernetes for managing containerized service instances
- **Monitoring and Logging**: Prometheus, Grafana, ELK Stack

## Deployment and Operations

- The service is deployed on a Kubernetes cluster, leveraging its auto-scaling capabilities for service instances.
- Continuous Integration and Continuous Deployment (CI/CD) pipelines are used for automated testing and deployment.
- Comprehensive monitoring and logging mechanisms are in place for real-time performance tracking and anomaly detection.

## Security Considerations

- All data in transit and at rest are encrypted using industry-standard encryption algorithms.
- Access to the service is secured using OAuth 2.0 for authentication and authorization.
- Regular security audits and vulnerability assessments are conducted.

## Conclusion

The User Profile Service is designed to be a robust, scalable, and secure microservice that can handle high loads and ensure consistent performance. By leveraging horizontal scaling strategies, modern technologies, and best practices in system design, this service can adapt to the growing demands of a large-scale web application.
"""

In [13]:
technical_model = ask_ai(example_content)

In [14]:
generate_graph(technical_model)

## Next steps

The entity model for the technical diagram can be made richer, with a focus on tools, function calls/data flow, storage and other components. This can be used to generate a technical diagram from a textual description. We encourage you to attempt this task as a challenge.